In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import csv
import time

Each drug has a fitness vector corresponding to the genetic state 0000, 0001, 0010, ...

In [ ]:
AMPf = np.array([1.851, 2.082, 1.948, 2.434, 2.024, 2.198, 2.033, 0.034, 1.57, 2.165, 0.051, 0.083, 2.186, 2.322, 0.088, 2.821])
AMf  = np.array([1.778, 1.782, 2.042, 1.752, 1.448, 1.544, 1.184, 0.063, 1.72, 2.008, 1.799, 2.005, 1.557, 2.247, 1.768, 2.047])
CECf = np.array([2.258, 1.996, 2.151, 2.648, 2.396, 1.846, 2.23, 0.214, 0.234, 0.172, 2.242, 0.093, 2.15, 0.095, 2.64, 0.516])
CTXf = np.array([0.16, 0.085, 1.936, 2.348, 1.653, 0.138, 2.295, 2.269, 0.185, 0.14, 1.969, 0.203, 0.225, 0.092, 0.119, 2.412])
ZOXf = np.array([0.993, 0.805, 2.069, 2.683, 1.698, 2.01, 2.138, 2.688, 1.106, 1.171, 1.894, 0.681, 1.116, 1.105, 1.103, 2.591])
CXMf = np.array([1.748, 1.7, 2.07, 1.938, 2.94, 2.173, 2.918, 3.272, 0.423, 1.578, 1.911, 2.754, 2.024, 1.678, 1.591, 2.923])
CROf = np.array([1.092, 0.287, 2.554, 3.042, 2.88, 0.656, 2.732, 0.436, 0.83, 0.54, 3.173, 1.153, 1.407, 0.751, 2.74, 3.227])
AMCf = np.array([1.435, 1.573, 1.061, 1.457, 1.672, 1.625, 0.073, 0.068, 1.417, 1.351, 1.538, 1.59, 1.377, 1.914, 1.307, 1.728])
CAZf = np.array([2.134, 2.656, 2.618, 2.688, 2.042, 2.756, 2.924, 0.251, 0.288, 0.576, 1.604, 1.378, 2.63, 2.677, 2.893, 2.563])
CTTf = np.array([2.125, 1.922, 2.804, 0.588, 3.291, 2.888, 3.082, 3.508, 3.238, 2.966, 2.883, 0.89, 0.546, 3.181, 3.193, 2.543])
SAMf = np.array([1.879, 2.533, 0.133, 0.094, 2.456, 2.437, 0.083, 0.094, 2.198, 2.57, 2.308, 2.886, 2.504, 3.002, 2.528, 3.453])
CPRf = np.array([1.743, 1.662, 1.763, 1.785, 2.018, 2.05, 2.042, 0.218, 1.553, 0.256, 0.165, 0.221, 0.223, 0.239, 1.811, 0.288])
CPDf = np.array([0.595, 0.245, 2.604, 3.043, 1.761, 1.471, 2.91, 3.096, 0.432, 0.388, 2.651, 1.103, 0.638, 0.986, 0.963, 3.268])
TZPf = np.array([2.679, 2.906, 2.427, 0.141, 3.038, 3.309, 2.528, 0.143, 2.709, 2.5, 0.172, 0.093, 2.453, 2.739, 0.609, 0.171])
FEPf = np.array([2.59, 2.572, 2.393, 2.832, 2.44, 2.808, 2.652, 0.611, 2.067, 2.446, 2.957, 2.633, 2.735, 2.863, 2.796, 3.203])

In [ ]:
# Assign State Binary Genotypes for labels
N = 4  # Bit Count of States
states = []
for i in range(16):
    b = format(i, 'b')
    bB = b.zfill(N)
    states.append(bB)

# Define the Hamming Distance
def hamDist(Si, Sj):
    return sum(ei != ej for ei, ej in zip(Si, Sj))

# Zero State of equal probabilities
S0 = np.array(np.ones(len(states))/2**N)

# Make a drug class
class Drug:
    # This will make it easy to call certain qualities
    # Arguments:
    # name is a string
    # Fit is an array of the fitness values
    # States is the binary genotype that corresponds to the bins
    # R is the biasing variable that we set to 0
    def __init__(self, name, Fit, states, LFP, R=0):
        # @par
        self.name = name
        self.Fit = np.array(Fit)
        self.states = states
        self.LFP = LFP
        self.R = R
        
        # The following code is to calculate the transition matrix
        P = np.zeros([len(self.states),len(self.states)])
    
        for i in range(len(self.states)):
            for j in range(len(self.states)):
                if self.Fit[j] >= self.Fit[i] and hamDist(self.states[i], self.states[j]) == 1:
                    P[i, j] = (self.Fit[j] - self.Fit[i])**R
                    d = 0
                    for s in range(len(self.states)):
                        if hamDist(self.states[i], self.states[s]) == 1 and self.Fit[s] > self.Fit[i]:
                            d = d + (self.Fit[s] - self.Fit[i])**R
                    P[i, j] = P[i, j]/d
                    
                    
        # if there is no probability to reach any other state,
        # there is 100% probability to remain in the initial state.
        for p in range(len(P)):
            spots = np.where(P[p] != 0)
            if len(spots[0])==0:
                P[p, p]=1

        self.tMat = np.matrix(P)
        self.LPFf = self.Fit[int(self.LFP, 2)]

In [ ]:
AMP = Drug('Ampicillin', AMPf, states, '0110')
AM = Drug('Amoxicillin', AMf, states, '0010')
CEC = Drug('Cefaclor', CECf, states, '0100')
CTX = Drug('Cefotaxime', CTXf, states, '1010')
ZOX = Drug('Ceftizoxime', ZOXf, states, '1001')
CXM = Drug('Cefuroxime', CXMf, states, '0100')
CRO = Drug('Ceftriaxone', CROf, states, '0100')
AMC = Drug('Amoxicillin+Clav', AMCf, states, '0100')
CAZ = Drug('Ceftazidime', CAZf, states, '0011')
CTT = Drug('Cefotetan', CTTf, states, '1101')
SAM = Drug('Ampicillin+Sulbactam', SAMf, states, '1111')
CPR = Drug('Cefprozil', CPRf, states, '0011')
CPD = Drug('Cefpodoxime', CPDf, states, '1010')
TZP = Drug('Piperacillin+Tazobactam', TZPf, states, '1000')
FEP = Drug('Cefepime', FEPf, states, '0000')

In [ ]:
# list of all drug Classes
allDrugs = [AMP, AM, CEC, CTX, ZOX , CXM, CRO, AMC, CAZ, CTT, SAM, CPR, CPD, TZP, FEP]

In [ ]:
col_names=['Final Drug', 'Steering Drug 1', 'Steering Drug 1 Amount', 'Steering Drug 2', 'Steering Drug 2 Amount',
           'Drug1:Drug2', 'Drug1+Drug2', 'Fitness of infinite apps', 'Lowest Fitness Achieved',
           'Maximum Simulated Drug Application', 'R-Value', 'epsilon', 'rho', 'theta']

## Exploring the Importance of Third Drug Application

Applying the third drug --- the steered drug --- provides the landscape after single or infinite applications. We can also look at the landscape just before applying the third drug.

We test up to 10 applications with tolerance $\epsilon$ defined as follows.

$$f_{\mathrm{simulate}}(\mathrm{NA, NB, NC})=\epsilon f_{\mathrm{inf}}(\infty, \infty, \mathrm{NC}) $$

NA, NB, and NC are the number of times a drug is applied.
NA and NB are ranges of 0 to 10 (applications) and NC is either 0, 1, or infinity.

First, we look at the fitness 
### Without applying the third drug

In [ ]:
start = time.time()
app = 10
epsilon = 0.05

filename = 'R0-10_AllDrugs_OnlyCountedWithin95perc_app10_eps_NoThirdInf.csv'
f = open(filename, 'w')
drugWriter = csv.writer(f, delimiter=',', lineterminator='\n')

for r in range(0, 11):

    AMP = Drug('Ampicillin', AMPf, states, '0110', R=r)
    AM = Drug('Amoxicillin', AMf, states, '0010', R=r)
    CEC = Drug('Cefaclor', CECf, states, '0100', R=r)
    CTX = Drug('Cefotaxime', CTXf, states, '1010', R=r)
    ZOX = Drug('Ceftizoxime', ZOXf, states, '1001', R=r)
    CXM = Drug('Cefuroxime', CXMf, states, '0100', R=r)
    CRO = Drug('Ceftriaxone', CROf, states, '0100', R=r)
    AMC = Drug('Amoxicillin+Clav', AMCf, states, '0100', R=r)
    CAZ = Drug('Ceftazidime', CAZf, states, '0011', R=r)
    CTT = Drug('Cefotetan', CTTf, states, '1101', R=r)
    SAM = Drug('Ampicillin+Sulbactam', SAMf, states, '1111', R=r)
    CPR = Drug('Cefprozil', CPRf, states, '0011', R=r)
    CPD = Drug('Cefpodoxime', CPDf, states, '1010', R=r)
    TZP = Drug('Piperacillin+Tazobactam', TZPf, states, '1000', R=r)
    FEP = Drug('Cefepime', FEPf, states, '0000', R=r)

    # list of all drug Classes
    allDrugs = [AMP, AM, CEC, CTX, ZOX , CXM, CRO, AMC, CAZ, CTT, SAM, CPR, CPD, TZP, FEP]
    # This iterates as the third drug
    for DRUG in allDrugs:
        # First Drug applied
        for i, DRUG1 in enumerate(allDrugs):
            # Second drug applied
            for j, DRUG2 in enumerate(allDrugs):
                # Make sure the drug sequence does not include the
                # final steering drug
                if DRUG1 != DRUG:
                    if DRUG2 != DRUG:
                        # Initialize a matrix
                        avgFitMat = np.zeros([app+1, app+1])
                        # f(inf, inf, inf) calculation
#                         fInfSN = np.array(S0) * DRUG1.tMat**100 * DRUG2.tMat**100 * DRUG.tMat**100
                        fInfSN = np.array(S0) * DRUG1.tMat**100 * DRUG2.tMat**100
                        fInfAvgFit = np.dot(np.array(fInfSN[0, :]), DRUG.Fit)
                        # Calculate f(NA, NB, inf)
                        for NA in range(0, app+1):
                            for NB in range(0, app+1):
#                                 SN = np.array(S0) * DRUG1.tMat**NA * DRUG2.tMat**NB * DRUG.tMat**100
                                SN = np.array(S0) * DRUG1.tMat**NA * DRUG2.tMat**NB
                                # Calculate the average fitness for NA, NB apps
                                avgFit = np.dot(np.array(SN[0, :]), DRUG.Fit)
                                # And put that value in a matrix
                                avgFitMat[NA, NB] = avgFit
                        # Now make the comparison to find
                        # f(NA, NB, inf) = epsilon * f(inf, inf, inf)
                        # on Second thought, this kind of does not make sense
                        # if the value is not a max, right...
#                         Z = np.where(avgFitMat <= (1+epsilon) * fInfAvgFit)
                        # Instead, f(NA, NB, inf) = (1+/-epsilon) * f(inf, inf, inf)
                        Z = np.where(np.logical_and(avgFitMat<=fInfAvgFit*(1+epsilon), avgFitMat>=fInfAvgFit*(1-epsilon)))
                        indx = [[r, c] for r in Z[0] for c in Z[1]]
                        if len(indx) == 0:
                            pass
                        else:
                            # Applied drug 1 x times
                            x = indx[0][0]
                            # Applied drug 2 y times
                            y = indx[0][1]
                            vals = [DRUG.name, DRUG1.name, x, DRUG2.name, y,
                                    x/y, x+y, fInfAvgFit, avgFitMat[x, y], app,
                                    DRUG.R, epsilon, np.sqrt(x**2 + y**2),
                                    np.arctan(y/x)]
                            drugWriter.writerow(vals)
end = time.time()
print(end-start)

Looking at the data as a dataframe for our familiar $SAM\rightarrow CPR \rightarrow AMP$.

In [ ]:
filename = 'R0-10_AllDrugs_OnlyCountedWithin95perc_app10_eps_NoThirdInf.csv'
df = pd.read_csv(filename, names=col_names)
df = df[df['Final Drug'] == AMP.name]
df = df[(df['Steering Drug 1'] == SAM.name) & (df['Steering Drug 2'] == CPR.name)]
df

### With third DRUG Application Infinite times

In [ ]:
start = time.time()
app = 10
epsilon = 0.05

filename = 'R0-10_AllDrugs_OnlyCountedWithin95perc_app10_eps_infThird.csv'
f = open(filename, 'w')
drugWriter = csv.writer(f, delimiter=',', lineterminator='\n')

for r in range(0, 11):

    AMP = Drug('Ampicillin', AMPf, states, '0110', R=r)
    AM = Drug('Amoxicillin', AMf, states, '0010', R=r)
    CEC = Drug('Cefaclor', CECf, states, '0100', R=r)
    CTX = Drug('Cefotaxime', CTXf, states, '1010', R=r)
    ZOX = Drug('Ceftizoxime', ZOXf, states, '1001', R=r)
    CXM = Drug('Cefuroxime', CXMf, states, '0100', R=r)
    CRO = Drug('Ceftriaxone', CROf, states, '0100', R=r)
    AMC = Drug('Amoxicillin+Clav', AMCf, states, '0100', R=r)
    CAZ = Drug('Ceftazidime', CAZf, states, '0011', R=r)
    CTT = Drug('Cefotetan', CTTf, states, '1101', R=r)
    SAM = Drug('Ampicillin+Sulbactam', SAMf, states, '1111', R=r)
    CPR = Drug('Cefprozil', CPRf, states, '0011', R=r)
    CPD = Drug('Cefpodoxime', CPDf, states, '1010', R=r)
    TZP = Drug('Piperacillin+Tazobactam', TZPf, states, '1000', R=r)
    FEP = Drug('Cefepime', FEPf, states, '0000', R=r)

    # list of all drug Classes
    allDrugs = [AMP, AM, CEC, CTX, ZOX , CXM, CRO, AMC, CAZ, CTT, SAM, CPR, CPD, TZP, FEP]
    # This iterates as the third drug
    for DRUG in allDrugs:
        # First Drug applied
        for i, DRUG1 in enumerate(allDrugs):
            # Second drug applied
            for j, DRUG2 in enumerate(allDrugs):
                # Make sure the drug sequence does not include the
                # final steering drug
                if DRUG1 != DRUG:
                    if DRUG2 != DRUG:
                        # Initialize a matrix
                        avgFitMat = np.zeros([app+1, app+1])
                        # f(inf, inf, inf) calculation
                        fInfSN = np.array(S0) * DRUG1.tMat**100 * DRUG2.tMat**100 * DRUG.tMat**100
                        fInfAvgFit = np.dot(np.array(fInfSN[0, :]), DRUG.Fit)
                        # Calculate f(NA, NB, inf)
                        for NA in range(0, app+1):
                            for NB in range(0, app+1):
                                # Fitness before third drug
                                SN = np.array(S0) * DRUG1.tMat**NA * DRUG2.tMat**NB * DRUG.tMat**100
                                # Calculate the average fitness for NA, NB apps
                                avgFit = np.dot(np.array(SN[0, :]), DRUG.Fit)
                                # And put that value in a matrix
                                avgFitMat[NA, NB] = avgFit
                        # Now make the comparison to find
                        # f(NA, NB, inf) = epsilon * f(inf, inf, inf)
                        # on Second thought, this kind of does not make sense
                        # if the value is not a max, right...
#                         Z = np.where(avgFitMat >= epsilon * fInfAvgFit)
                        # Instead, f(NA, NB, inf) = (1+/-epsilon) * f(inf, inf, inf)
                        Z = np.where(np.logical_and(avgFitMat<=fInfAvgFit*(1+epsilon), avgFitMat>=fInfAvgFit*(1-epsilon)))
                        indx = [[r, c] for r in Z[0] for c in Z[1]]
                        if len(indx) == 0:
                            pass
                        else:
                            # Applied drug 1 x times
                            x = indx[0][0]
                            # Applied drug 2 y times
                            y = indx[0][1]
                            vals = [DRUG.name, DRUG1.name, x, DRUG2.name, y,
                                    x/y, x+y, fInfAvgFit, avgFitMat[x, y], app,
                                    DRUG.R, epsilon, np.sqrt(x**2 + y**2),
                                    np.arctan(y/x)]
                            drugWriter.writerow(vals)
end = time.time()
print(end-start)

In [ ]:
filename = 'R0-10_AllDrugs_OnlyCountedWithin95perc_app10_eps_infThird.csv'
df = pd.read_csv(filename, names=col_names)
df = df[df['Final Drug'] == AMP.name]
df = df[(df['Steering Drug 1'] == SAM.name) & (df['Steering Drug 2'] == CPR.name)]
df

### With third DRUG Application One Time

In [ ]:
start = time.time()
app = 10
epsilon = 0.05

filename = 'R0-10_AllDrugs_OnlyCountedWithin95perc_app10_eps_onceThird.csv'
f = open(filename, 'w')
drugWriter = csv.writer(f, delimiter=',', lineterminator='\n')

for r in range(0, 11):

    AMP = Drug('Ampicillin', AMPf, states, '0110', R=r)
    AM = Drug('Amoxicillin', AMf, states, '0010', R=r)
    CEC = Drug('Cefaclor', CECf, states, '0100', R=r)
    CTX = Drug('Cefotaxime', CTXf, states, '1010', R=r)
    ZOX = Drug('Ceftizoxime', ZOXf, states, '1001', R=r)
    CXM = Drug('Cefuroxime', CXMf, states, '0100', R=r)
    CRO = Drug('Ceftriaxone', CROf, states, '0100', R=r)
    AMC = Drug('Amoxicillin+Clav', AMCf, states, '0100', R=r)
    CAZ = Drug('Ceftazidime', CAZf, states, '0011', R=r)
    CTT = Drug('Cefotetan', CTTf, states, '1101', R=r)
    SAM = Drug('Ampicillin+Sulbactam', SAMf, states, '1111', R=r)
    CPR = Drug('Cefprozil', CPRf, states, '0011', R=r)
    CPD = Drug('Cefpodoxime', CPDf, states, '1010', R=r)
    TZP = Drug('Piperacillin+Tazobactam', TZPf, states, '1000', R=r)
    FEP = Drug('Cefepime', FEPf, states, '0000', R=r)

    # list of all drug Classes
    allDrugs = [AMP, AM, CEC, CTX, ZOX , CXM, CRO, AMC, CAZ, CTT, SAM, CPR, CPD, TZP, FEP]
    # This iterates as the third drug
    for DRUG in allDrugs:
        # First Drug applied
        for i, DRUG1 in enumerate(allDrugs):
            # Second drug applied
            for j, DRUG2 in enumerate(allDrugs):
                # Make sure the drug sequence does not include the
                # final steering drug
                if DRUG1 != DRUG:
                    if DRUG2 != DRUG:
                        # Initialize a matrix
                        avgFitMat = np.zeros([app+1, app+1])
                        # f(inf, inf, inf) calculation
                        fInfSN = np.array(S0) * DRUG1.tMat**100 * DRUG2.tMat**100 * DRUG.tMat**100
                        fInfAvgFit = np.dot(np.array(fInfSN[0, :]), DRUG.Fit)
                        # Calculate f(NA, NB, inf)
                        for NA in range(0, app+1):
                            for NB in range(0, app+1):
                                # Fitness before third drug
                                SN = np.array(S0) * DRUG1.tMat**NA * DRUG2.tMat**NB * DRUG.tMat**1
                                # Calculate the average fitness for NA, NB apps
                                avgFit = np.dot(np.array(SN[0, :]), DRUG.Fit)
                                # And put that value in a matrix
                                avgFitMat[NA, NB] = avgFit
                        # Now make the comparison to find
                        # f(NA, NB, inf) = epsilon * f(inf, inf, inf)
                        # on Second thought, this kind of does not make sense
                        # if the value is not a max, right...
#                         Z = np.where(avgFitMat >= epsilon * fInfAvgFit)
                        # Instead, f(NA, NB, inf) = (1+/-epsilon) * f(inf, inf, inf)
                        Z = np.where(np.logical_and(avgFitMat<=fInfAvgFit*(1+epsilon), avgFitMat>=fInfAvgFit*(1-epsilon)))
                        indx = [[r, c] for r in Z[0] for c in Z[1]]
                        if len(indx) == 0:
                            pass
                        else:
                            # Applied drug 1 x times
                            x = indx[0][0]
                            # Applied drug 2 y times
                            y = indx[0][1]
                            vals = [DRUG.name, DRUG1.name, x, DRUG2.name, y,
                                    x/y, x+y, fInfAvgFit, avgFitMat[x, y], app,
                                    DRUG.R, epsilon, np.sqrt(x**2 + y**2),
                                    np.arctan(y/x)]
                            drugWriter.writerow(vals)
end = time.time()
print(end-start)

In [ ]:
filename = 'R0-10_AllDrugs_OnlyCountedWithin95perc_app10_eps_onceThird.csv'
df = pd.read_csv(filename, names=col_names)
df = df[df['Final Drug'] == AMP.name]
df = df[(df['Steering Drug 1'] == SAM.name) & (df['Steering Drug 2'] == CPR.name)]
df

### Short Discussion

Without applying the third drug, I do not know what value to attempt to reach. In showing asymmetry, we have suggested that we can approach the lowest fitness peak with different weights from two steering drugs.

The infinite and single applications of the final drug make sense to show that we change the effect of the final drug based on the application of two steering drugs.

At first, I thought that maybe 10 applications was not enough to show how the fitness matrix changes before applying the final drug; however, we can investigate very briefly the difference in applying the third drug by showing the SAM CPR AMP progression.

In [ ]:
fig, axes = plt.subplots(ncols=3, nrows=1, figsize=(18,10))

Ap = 20
avgMat = np.zeros([Ap+1, Ap+1])
for NA in range(0,Ap+1):
    for NB in range(0,Ap+1):
        SN = np.array(S0) * SAM.tMat**NA * CPR.tMat**NB
        AvgFit = np.dot(np.array(SN[0,:]), AMP.Fit)
        avgMat[NA, NB] = AvgFit
cmap = plt.cm.inferno
norm = mpl.colors.BoundaryNorm(np.arange(0,3.4,0.2), cmap.N)
ax = axes[0]
Z0 = ax.matshow(avgMat, cmap=cmap, norm=norm)
ax.set_xlabel('CPR Applied NB Times')
ax.set_ylabel('SAM Applied NA Times')
ax.set_title('Average Fitness <f> without applying AMP')

avgMat = np.zeros([Ap+1, Ap+1])
for NA in range(0,Ap+1):
    for NB in range(0,Ap+1):
        SN = np.array(S0) * SAM.tMat**NA * CPR.tMat**NB * AMP.tMat**100
        AvgFit = np.dot(np.array(SN[0,:]), AMP.Fit)
        avgMat[NA, NB] = AvgFit
cmap = plt.cm.inferno
norm = mpl.colors.BoundaryNorm(np.arange(0,3.4,0.2), cmap.N)
ax = axes[1]
Z = ax.matshow(avgMat, cmap=cmap, norm=norm)
ax.set_xlabel('CPR Applied NB Times')
ax.set_ylabel('SAM Applied NA Times')
ax.set_title('Average Fitness <f> Applying AMP $\infty$ Times')

avgMat = np.zeros([Ap+1, Ap+1])
for NA in range(0,Ap+1):
    for NB in range(0,Ap+1):
        SN = np.array(S0) * SAM.tMat**NA * CPR.tMat**NB * AMP.tMat
        AvgFit = np.dot(np.array(SN[0,:]), AMP.Fit)
        avgMat[NA, NB] = AvgFit
cmap = plt.cm.inferno
norm = mpl.colors.BoundaryNorm(np.arange(0,3.4,0.2), cmap.N)
ax = axes[2]
Z = ax.matshow(avgMat, cmap=cmap, norm=norm)
ax.set_xlabel('CPR Applied NB Times')
ax.set_ylabel('SAM Applied NA Times')
ax.set_title('Average Fitness <f> without applying AMP Once')
fig.colorbar(Z0,  ax=axes.ravel().tolist(), orientation='horizontal')
plt.show()

This shows that increasing the application amount beyond 10 to 20 does not affect the values in the long run. In fact this suggests that we only have to look at a very small subset of applications.

But, to the point of what we are looking for, without applying the final drug, there is a global minimum at close to zero. This, however, is not the value of $f_\mathrm{inf}(\infty, \infty, \infty)$. In this case, the value of $f_\mathrm{inf}$ is 2.033. This global minimum is important, and it is likely that the same relationship to reaching $f_\mathrm{inf}$.

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.matshow(AMP.tMat)

## Applying New Concepts and defining $\rho$ and $\phi$.

$\rho$ decreasing with R and flattening out

$\phi$ dependence on R

Nice graph:

Polar where every combo is a $\rho$ vs $\phi$ scatter plot.

Each plot different for R = 0, 1, 2...

Can $\rho$ or $\phi$ be predicted from the structure of the drug matrix.

Keyword: eigenspectrum

$f_{ABC}(x, y, \infty) = \epsilon f_{ABC}(\infty, \infty, \infty)$